In [3]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

########################
## Check CLI commands ##
########################

The following required programs are available in the shell:
  * jq
  * json2yaml
  * marlowe-cli
  * marlowe-runtime-cli
  * cardano-cli
  * cardano-address
  * cardano-wallet

#########################
## Check required envs ##
#########################

The following environment variables are available in the shell:
  * CARDANO_NODE_SOCKET_PATH = /ipc/node.socket
  * MARLOWE_RT_HOST = proxy
  * MARLOWE_RT_PORT = 3700
  * MARLOWE_RT_WEBSERVER_HOST = web-server
  * MARLOWE_RT_WEBSERVER_PORT = 3780
  * MARLOWE_RT_WEBSERVER_URL = http://web-server:3780
  * FAUCET_ADDR = addr_test1vr83hwnap8hrglw59e5f6xhq4lsz9uyejtkt7f846g0893g0a74d5
  * FAUCET_SKEY = ../../keys/faucet.skey

###################
## Check Network ##
###################

The NETWORK is set to preprod
CARDANO_TESTNET_MAGIC = 1
CARDANO_SCAN_URL = https://preprod.cardanoscan.io
MARLOWE_SCAN_URL = https://preprod.marlowescan.com


In [4]:
ALICE_SKEY=$KEYS/lender.skey
ALICE_ADDR=$(cat $KEYS/lender.address)
echo "ALICE_ADDR = $ALICE_ADDR"

ALICE_ADDR = addr_test1vp25zqq382ewtkafmejeu37m7qjwcly8s5rm4nzc3vzavqcnq90w5


In [5]:
BOB_SKEY=$KEYS/borrower.skey
BOB_ADDR=$(cat $KEYS/borrower.address)
echo "BOB_ADDR = $BOB_ADDR"

BOB_ADDR = addr_test1vp9lcx33v29jvenwaj8n20mhqy8v8dckj7yp055707egw7ge7c30c


In [6]:
CHARLIE_SKEY=$KEYS/mediator.skey
CHARLIE_ADDR=$(cat $KEYS/mediator.address)
echo "CHARLIE_ADDR = $CHARLIE_ADDR"

CHARLIE_ADDR = addr_test1vqcr8jzg0p5wnrjnlfq5jq905je4snx6ze4m8nk62yyjuccec5lke


In [ ]:
$SCRIPTS/fund-wallets.sh $ALICE_ADDR $BOB_ADDR $CHARLIE_ADDR

In [13]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2     1        1000000000 lovelace + TxOutDatumNone


In [14]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BOB_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2     2        1000000000 lovelace + TxOutDatumNone


In [15]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CHARLIE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2     3        1000000000 lovelace + TxOutDatumNone


In [16]:
json2yaml lottery-contract.json

timeout: 1696464000000
timeout_continuation: close
when:
- case:
    deposits: 100000000
    into_account:
      role_token: Alice
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Alice
  then:
    timeout: 1696464000000
    timeout_continuation: close
    when:
    - case:
        deposits: 100000000
        into_account:
          role_token: Bob
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Bob
      then:
        timeout: 1696467600000
        timeout_continuation: close
        when:
        - case:
            choose_between:
            - from: 1
              to: 2
            for_choice:
              choice_name: winner
              choice_owner:
                role_token: Charlie
          then:
            else:
              else: close
              if:
                equal_to: 2
                value:
                  value_of_choice:
                    choice_na

In [28]:
MIN_LOVELACE=2000000

In [29]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat lottery-contract.json`
roles:
  Alice: "$ALICE_ADDR"
  Bob: "$BOB_ADDR"
  Charlie: "$CHARLIE_ADDR"
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json | jq


{
  "contract": {
    "timeout": 1696464000000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 100000000,
          "into_account": {
            "role_token": "Alice"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": ""
          },
          "party": {
            "role_token": "Alice"
          }
        },
        "then": {
          "timeout": 1696464000000,
          "timeout_continuation": "close",
          "when": [
            {
              "case": {
                "deposits": 100000000,
                "into_account": {
                  "role_token": "Bob"
                },
                "of_token": {
                  "currency_symbol": "",
                  "token_name": ""
                },
                "party": {
                  "role_token": "Bob"
                }
              },
              "then": {
                "timeout": 1696467600000,
            

          }
        },
        "then": {
          "timeout": 1696464000000,
          "timeout_continuation": "close",
          "when": [
            {
              "case": {
                "deposits": 100000000,
                "into_account": {
                  "role_token": "Alice"
                },
                "of_token": {
                  "currency_symbol": "",
                  "token_name": ""
                },
                "party": {
                  "role_token": "Alice"
                }
              },
              "then": {
                "timeout": 1696467600000,
                "timeout_continuation": "close",
                "when": [
                  {
                    "case": {
                      "choose_between": [
                        {
                          "from": 1,
                          "to": 2
                        }
                      ],
                      "for_choice": {
                        "choice_name": "winn

In [30]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $CHARLIE_ADDR" \b
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6%231
resource:
  contractId: 1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6#1
  safetyErrors: []
  txBody:
    cborHex: 86a80081825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2030d81825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2030185a200581d603033c8487868e98e53fa414900afa4b3584cda166bb3ceda51092e63011a3b45c6d1a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e8480028200582034f97928d9dd232d3ee512c4e7174288dcdbb9aa4ee41a731b9ee8d3f953ad87a200581d60554100113ab2e5dba9de659e47dbf024ec7c878507bacc588b05d60301821a000fb7caa1581cd86a6097ad1b472d5aac01472d5e7a6ce20c91e46187e9c5b02e8aa1a145416c69636501a200581d604bfc1a31628b26666eec8f353f77010ec3b716978817d29e7fb2877901821a000f961ea1581cd86a6097ad1b472d5aac01472d5e7a6ce20c91e46187e9c5b02e8aa1a143426f6201a200581d603033c8487868e98e53fa414900afa4b3584cda166bb3ceda51092e630

    description: ''
    type: TxBodyBabbage


In [31]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6#1


In [32]:
jq '.resource.txBody' response-1.json > tx-1.unsigned
du -hs tx-1.unsigned

12K	tx-1.unsigned


In [33]:
echo "submiting transaction..."
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$CHARLIE_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

submiting transaction...
TX_1 = 1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6


In [34]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_1
echo
echo "Marlowe Scan (high level)"
$SCRIPTS/marlowe-scan.sh $CONTRACT_ID

Cardano Scan (low level)
https://preprod.cardanoscan.io/transaction/1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6?tab=summary

Marlowe Scan (high level)
https://preprod.marlowescan.com/contractView?tab=info&contractId=1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6%231


In [38]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://web-server:3780/contracts/1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6%231


In [39]:
marlowe-cli input deposit \
  --deposit-party Bob \
  --deposit-account Bob \
  --deposit-amount 100000000 \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Bob
into_account:
  role_token: Bob
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 100000000


In [40]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Bob"},"into_account":{"role_token":"Bob"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":100000000}],"metadata":{},"tags":{},"version":"v1"}


In [41]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BOB_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

links:
  transaction: contracts/1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6%231/transactions/38b1d664c1334b9ebdb0bf7520332b9cef7f354bc40a49ab3c2c656f311ef820
resource:
  contractId: 1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6#1
  transactionId: 38b1d664c1334b9ebdb0bf7520332b9cef7f354bc40a49ab3c2c656f311ef820
  txBody:
    cborHex: 86aa00838258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6018258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e603825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2020d81825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a20212818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d604bfc1a31628b26666eec8f353f77010ec3b716978817d29e7fb28779011a35985700a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a0614658002820058205257a026403df1202f04b8a3bb4d69a3115e2b37aaefafa0b837896dd0d70700a200581d604bfc1

In [42]:
jq '.resource.txBody' response-2.json > tx-2.unsigned
du -hs tx-2.unsigned

4.0K	tx-2.unsigned


In [44]:
echo "submiting transaction..."
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$BOB_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

submiting transaction...
TX_2 = 38b1d664c1334b9ebdb0bf7520332b9cef7f354bc40a49ab3c2c656f311ef820


In [39]:
marlowe-cli input deposit \
  --deposit-party Bob \
  --deposit-account Bob \
  --deposit-amount 100000000 \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Bob
into_account:
  role_token: Bob
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 100000000


In [40]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Bob"},"into_account":{"role_token":"Bob"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":100000000}],"metadata":{},"tags":{},"version":"v1"}


In [41]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BOB_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

links:
  transaction: contracts/1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6%231/transactions/38b1d664c1334b9ebdb0bf7520332b9cef7f354bc40a49ab3c2c656f311ef820
resource:
  contractId: 1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6#1
  transactionId: 38b1d664c1334b9ebdb0bf7520332b9cef7f354bc40a49ab3c2c656f311ef820
  txBody:
    cborHex: 86aa00838258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6018258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e603825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2020d81825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a20212818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d604bfc1a31628b26666eec8f353f77010ec3b716978817d29e7fb28779011a35985700a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a0614658002820058205257a026403df1202f04b8a3bb4d69a3115e2b37aaefafa0b837896dd0d70700a200581d604bfc1

In [42]:
jq '.resource.txBody' response-2.json > tx-2.unsigned
du -hs tx-2.unsigned

4.0K	tx-2.unsigned


In [44]:
echo "submiting transaction..."
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$BOB_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

submiting transaction...
TX_2 = 38b1d664c1334b9ebdb0bf7520332b9cef7f354bc40a49ab3c2c656f311ef820


In [45]:
marlowe-cli input deposit \
  --deposit-party Alice \
  --deposit-account Alice \
  --deposit-amount 100000000 \
  --out-file input-3.json
json2yaml input-3.json

input_from_party:
  role_token: Alice
into_account:
  role_token: Alice
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 100000000


In [46]:
yaml2json << EOI > request-3.json
version: v1
inputs: [$(cat input-3.json)]
metadata: {}
tags: {}
EOI
cat request-3.json

{"inputs":[{"input_from_party":{"role_token":"Alice"},"into_account":{"role_token":"Alice"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":100000000}],"metadata":{},"tags":{},"version":"v1"}


In [47]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ALICE_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

links:
  transaction: contracts/1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6%231/transactions/ddc975687728e532d068301936afa6753eda181018f4e426008c592ce8682ed9
resource:
  contractId: 1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6#1
  transactionId: ddc975687728e532d068301936afa6753eda181018f4e426008c592ce8682ed9
  txBody:
    cborHex: 86aa00838258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e60282582038b1d664c1334b9ebdb0bf7520332b9cef7f354bc40a49ab3c2c656f311ef82001825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a2010d81825820db06746aa07aaaa17e52fc7fbf39be49dea9bf544deed2ea5f4d003acfa645a20112818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d60554100113ab2e5dba9de659e47dbf024ec7c878507bacc588b05d603011a359949b5a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a0c0a46800282005820205b9216c510766aec79acb84875a0a1007454b57205e34b4b871f8a2b7576b8a200581d6055410

In [48]:
jq '.resource.txBody' response-3.json > tx-3.unsigned
du -hs tx-3.unsigned

4.0K	tx-3.unsigned


In [49]:
echo "submiting transaction..."
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$ALICE_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

submiting transaction...
TX_3 = 


In [54]:
marlowe-cli input choose \
  --choice-name winner \
  --choice-party Charlie \
  --choice-number 1 \
  --out-file input-4.json
json2yaml input-4.json

for_choice_id:
  choice_name: winner
  choice_owner:
    role_token: Charlie
input_that_chooses_num: 1


In [55]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
tags: {}
EOI
cat request-4.json

{"inputs":[{"for_choice_id":{"choice_name":"winner","choice_owner":{"role_token":"Charlie"}},"input_that_chooses_num":1}],"metadata":{},"tags":{},"version":"v1"}


In [56]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $CHARLIE_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

links:
  transaction: contracts/1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6%231/transactions/7d8e65c6dff0b1852f3b258228fb82756b8cab7025b4d14a6f0c398b213a58a1
resource:
  contractId: 1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6#1
  transactionId: 7d8e65c6dff0b1852f3b258228fb82756b8cab7025b4d14a6f0c398b213a58a1
  txBody:
    cborHex: 86aa00838258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6008258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e604825820ddc975687728e532d068301936afa6753eda181018f4e426008c592ce8682ed9010d818258201c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e60012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010184a200581d603033c8487868e98e53fa414900afa4b3584cda166bb3ceda51092e63011a3b39925ca200581d603033c8487868e98e53fa414900afa4b3584cda166bb3ceda51092e6301821a000fd976a1581cd86a6097ad1b472d5aac01472d5e7a6ce20c91e46187e9c5b02e8aa1a147436861726c696501a3005

In [57]:
jq '.resource.txBody' response-4.json > tx-4.unsigned
du -hs tx-4.unsigned

4.0K	tx-4.unsigned


In [58]:
echo "submiting transaction..."
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$CHARLIE_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

submiting transaction...
TX_4 = 7d8e65c6dff0b1852f3b258228fb82756b8cab7025b4d14a6f0c398b213a58a1


In [59]:
yaml2json << EOI > request-5.json
contractId: "$CONTRACT_ID"
role: Alice
EOI
cat request-5.json | jq

{
  "contractId": "1c18d683699867980269d9c205d6f9e87b4494fb8bf28f69606ec019fc1ab1e6#1",
  "role": "Alice"
}


In [60]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ALICE_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

links:
  withdrawal: withdrawals/a0399bb51588da62874416db15b795bf2f2d681d7759ec93a7323cc7ce71ddbd
resource:
  txBody:
    cborHex: 86a800838258207d8e65c6dff0b1852f3b258228fb82756b8cab7025b4d14a6f0c398b213a58a102825820ddc975687728e532d068301936afa6753eda181018f4e426008c592ce8682ed900825820ddc975687728e532d068301936afa6753eda181018f4e426008c592ce8682ed9020d81825820ddc975687728e532d068301936afa6753eda181018f4e426008c592ce8682ed90012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d60554100113ab2e5dba9de659e47dbf024ec7c878507bacc588b05d603011a417fed0da200581d60554100113ab2e5dba9de659e47dbf024ec7c878507bacc588b05d60301821a000fb7caa1581cd86a6097ad1b472d5aac01472d5e7a6ce20c91e46187e9c5b02e8aa1a145416c6963650110a200581d60554100113ab2e5dba9de659e47dbf024ec7c878507bacc588b05d603011a35919bb9111a0007adfc021a00051ea80b5820d25f6790af25771dd174ea89debcc3e8f867a47b331a99e153cf031a8943f57d9fff81d8799f581cd86a6097ad1b472d5aac01472d5e7a6ce20c91e46187e9c5b02e8aa145416c6

In [61]:
jq '.resource.txBody' response-5.json > tx-5.unsigned
du -hs tx-5.unsigned

4.0K	tx-5.unsigned


In [62]:
echo "submiting transaction..."
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$ALICE_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

submiting transaction...
TX_5 = a0399bb51588da62874416db15b795bf2f2d681d7759ec93a7323cc7ce71ddbd
